In [ ]:
!pip install boto3 pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.5/215.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 11.3 MB/s eta 0:00:00


In [ ]:
!unzip /content/pets.zip -d /content

Archive:  /content/pets.zip
   creating: /content/pets/cat/
  inflating: /content/pets/cat/Image_1.jpg  
  inflating: /content/pets/cat/Image_10.jpg  
  inflating: /content/pets/cat/Image_11.jpg  
  inflating: /content/pets/cat/Image_12.jpg  
  inflating: /content/pets/cat/Image_2.jpg  
  inflating: /content/pets/cat/Image_3.jpg  
  inflating: /content/pets/cat/Image_4.jpg  
  inflating: /content/pets/cat/Image_5.jpg  
  inflating: /content/pets/cat/Image_6.jpg  
  inflating: /content/pets/cat/Image_7.jpeg  
  inflating: /content/pets/cat/Image_8.jpg  
  inflating: /content/pets/cat/Image_9.jpg  
   creating: /content/pets/dog/
  inflating: /content/pets/dog/Image_1_dog.jpg  
  inflating: /content/pets/dog/Image_10_dog.jpg  
  inflating: /content/pets/dog/Image_11_dog.jpg  
  inflating: /content/pets/dog/Image_12_dog.jpg  
  inflating: /content/pets/dog/Image_2_dog.jpg  
  inflating: /content/pets/dog/Image_3_dog.jpg  
  inflating: /content/pets/dog/Image_4_dog.jpg  
  inflating: /cont

In [ ]:
import json
import boto3
import base64
import os

bedrock_runtime=boto3.client('bedrock-runtime',region_name='us-east-1',aws_access_key_id='{}',aws_secret_access_key='{}')

from pinecone import Pinecone

pc = Pinecone(api_key="{}")
index = pc.Index("{}")

from io import BytesIO
from PIL import Image

MAX_IMAGE_HEIGHT: int = 2048
MAX_IMAGE_WIDTH: int = 2048

def resizeandGetByteData(imageFile):
    image = Image.open(imageFile)
    if (image.size[0] * image.size[1]) > (MAX_IMAGE_HEIGHT * MAX_IMAGE_WIDTH):
        image = image.resize((MAX_IMAGE_HEIGHT, MAX_IMAGE_WIDTH))
    with BytesIO() as output:
        image.save(output, 'png')
        bytes_data = output.getvalue()
    return bytes_data


def image_embedding(image_name):
  bytes_data = resizeandGetByteData(image_name)
  input_image = base64.b64encode(bytes_data).decode('utf8')
  body = json.dumps(
    {
        "inputImage": input_image
    }
  )
  response = bedrock_runtime.invoke_model(
  body=body,
  modelId="amazon.titan-embed-image-v1",
  accept="application/json",
  contentType="application/json"
  )
  response_body = json.loads(response.get("body").read())
  return response_body.get("embedding")

def get_image_files_from_directory(directory):
    return [os.path.join(directory, file) for file in os.listdir(directory)]

# Directory containing the images of cats
image_directory = '/content/pets/cat'

# Get list of image files in the cat directory
data_to_store = get_image_files_from_directory(image_directory)

print("Storing Cats data : ",data_to_store)

vectors=[]
for i in data_to_store:
  vectors.append({
            "id":i,
            "values": image_embedding(i),
            "metadata": {"category": "Cat"}
        })

index.upsert(
    vectors=vectors,
    namespace= "ns1"
)



# Directory containing the images of dogs
image_directory = '/content/pets/dog'

# Get list of image files in the dog directory
data_to_store = get_image_files_from_directory(image_directory)

print("Storing Dogs data : ",data_to_store)

vectors=[]
for i in data_to_store:
  vectors.append({
            "id":i,
            "values": image_embedding(i),
            "metadata": {"category": "Dog"}
        })

index.upsert(
    vectors=vectors,
    namespace= "ns1"
)

Storing Cats data :  ['/content/pets/cat/Image_12.jpg', '/content/pets/cat/Image_2.jpg', '/content/pets/cat/Image_5.jpg', '/content/pets/cat/Image_6.jpg', '/content/pets/cat/Image_8.jpg', '/content/pets/cat/Image_9.jpg', '/content/pets/cat/Image_1.jpg', '/content/pets/cat/Image_7.jpeg', '/content/pets/cat/Image_11.jpg', '/content/pets/cat/Image_3.jpg', '/content/pets/cat/Image_4.jpg', '/content/pets/cat/Image_10.jpg']
Storing Dogs data :  ['/content/pets/dog/Image_7_dog.jpg', '/content/pets/dog/Image_12_dog.jpg', '/content/pets/dog/Image_2_dog.jpg', '/content/pets/dog/Image_3_dog.jpg', '/content/pets/dog/Image_1_dog.jpg', '/content/pets/dog/Image_6_dog.jpg', '/content/pets/dog/Image_8_dog.jpg', '/content/pets/dog/Image_5_dog.jpg', '/content/pets/dog/Image_11_dog.jpg', '/content/pets/dog/Image_4_dog.jpg', '/content/pets/dog/Image_10_dog.jpg', '/content/pets/dog/Image_9_dog.jpg']


{'upserted_count': 12}

In [ ]:
classification=index.query(
    namespace="ns1",
    vector=image_embedding('/content/test1_pet.JPG'),
    include_metadata=True,
    top_k=1
)
#print(classification)
classification['matches'][0]['metadata']['category']

'Dog'

In [ ]:
classification=index.query(
    namespace="ns1",
    vector=image_embedding('/content/test1_pet2.JPG'),
    include_metadata=True,
    top_k=1
)
classification['matches'][0]['metadata']['category']

'Dog'

In [ ]:
classification=index.query(
    namespace="ns1",
    vector=image_embedding('/content/test5.JPG'),
    include_metadata=True,
    top_k=1
)
print(classification)
classification['matches'][0]['metadata']['category']

{'matches': [{'id': '/content/pets/dog/Image_8_dog.jpg',
              'metadata': {'category': 'Dog'},
              'score': 0.411355346,
              'values': []}],
 'namespace': 'ns1',
 'usage': {'read_units': 6}}


'Dog'